In [ ]:
# Importing Libraries
import time
import cv2
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
model = load_model("mymodel.h5", compile=False)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [ ]:
#Load the  Dictionary

with open('./category2label.pkl', 'rb') as cat:
    category_2_label = pickle.load(cat)

In [ ]:
#Defining Image Size

print(category_2_label)

img_size = (100, 100)

#Setting Box Colors

box_colors = {0: (0, 0, 255), 1: (0, 255, 0)}

In [ ]:
#Initializing the Face Cascade Classifier

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')



In [ ]:
# video from webcam

cap = cv2.VideoCapture(0)    #video capture object
begin_time = time.time()     #set to the current time, and frame_cnt is initialized to 0.
frame_cnt = 1

while cap.isOpened():        #captures a frame of video from the camera and processes it
    rate, frame = cap.read()  #The rate and frame values are obtained using the cap.read() method

    if not rate:    #check kore picture capture hoise kina , nah hole break
        break

    frame_cnt += 1      #The frame counter is incremented by one.
    frame = cv2.flip(frame, 1)
    face = face_cascade.detectMultiScale(frame, 1.5, 5)

    for (x, y, w, h) in face:
        # prediction

        roi = frame[y:y+h, x:x+w]
        data = cv2.resize(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB), img_size)
        data = data / 255.
        data = data.reshape((1,) + data.shape)

        pred = model.predict(data)

        target = int(round(pred[0][0]))

        # drawing box
        cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=box_colors[target], thickness=2)
        text = "{}: {:.2f}".format(category_2_label[target], pred[0][0])
        cv2.putText(frame, text, org=(x, y-5), fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=0.6, color=(255, 255, 255), thickness=1)

    endtime = time.time()-begin_time
    fps = frame_cnt / endtime


    #Displaying FPS
    
    cv2.putText(img=frame, text="FPS: "+str(round(fps, 2)), org=(10, 20),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,  fontScale=0.5, color=(255, 255, 255), thickness=1)
    # Show the frame
    cv2.imshow('Face Mask Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
